# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/output_data/cityweatherdata.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,City,Country,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed (MPH),Latitude,Longitude
0,san patricio,MX,78.60,79,23,4.92,19.22,-104.70
1,bredasdorp,ZA,59.00,77,70,11.41,-34.53,20.04
2,kindersley,CA,55.40,93,75,10.29,51.47,-109.17
3,punta arenas,CL,21.20,85,0,9.17,-53.15,-70.92
4,faanui,PF,77.47,79,50,23.31,-16.48,-151.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity (%)"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity (%)"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temperature"] < 80) & (filtered_weather_df["Max Temperature"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed (MPH)"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness (%)"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Country,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed (MPH),Latitude,Longitude
139,richards bay,ZA,75.56,47,0,9.64,-28.78,32.04
219,letlhakane,BW,76.35,16,0,9.04,-21.42,25.58
236,port hedland,AU,73.40,46,0,6.93,-20.32,118.57
250,liuli,TZ,79.21,57,0,6.51,-11.08,34.63
332,damietta,EG,75.20,88,0,9.17,31.42,31.81
337,shakawe,BW,78.01,17,0,6.64,-18.37,21.85
349,nosy varika,MG,77.50,63,0,7.27,-20.58,48.53
427,masvingo,ZW,73.40,25,0,9.17,-20.06,30.83
458,broome,AU,77.00,53,0,9.17,-17.97,122.23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 139: richards bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 219: letlhakane.
Closest hotel is Motswedi Hotel P650.
------------
Retrieving Results for Index 236: port hedland.
Closest hotel is The Esplanade Hotel.
------------
Retrieving Results for Index 250: liuli.
Closest hotel is Josephs Dream Camp Site Beach.
------------
Retrieving Results for Index 332: damietta.
Closest hotel is السيد السيد بدر للكراسي.
------------
Retrieving Results for Index 337: shakawe.
Closest hotel is Ditlhapi Guest House.
------------
Retrieving Results for Index 349: nosy varika.
Closest hotel is Hotel Volazara.
------------
Retrieving Results for Index 427: masvingo.
Closest hotel is River Side Lodge.
------------
Retrieving Results for Index 458: broome.
Closest hotel is Mangrove Hotel.
------------


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))